# Run single CBF with bias with specified dataset and parameters

In [ ]:
pwd

In [ ]:
cd ..

#### Load libs

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pyplot
%matplotlib inline
import scipy.sparse as sps

In [ ]:
from Data_manager.DataReader_ImportAll import *
import multiprocessing, traceback, os
from functools import  partial
# TODO ADDED
import skopt
import datetime, time
# TODO /ADDED

from ParameterTuning.SearchAbstractClass import SearchInputRecommenderParameters
from ParameterTuning.run_parameter_search import runParameterSearch_Content, runParameterSearch_Collaborative
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.DataSplitter_leave_k_out import DataSplitter_leave_k_out

from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender


In [ ]:
dataset_class = Movielens20MReader # Movielens10MReader # CiteULike_aReader
feature_weighting = "none" # "TF-IDF" # "BM25" # 
allow_bias_ICM = False
ICM_bias = None # None or float

In [ ]:
similarity_type_list = ["cosine"]

output_folder = "result_experiments/{}".format(dataset_class.DATASET_SUBFOLDER)
if allow_bias_ICM:
    output_folder += "ICM_bias/"
else:
    output_folder += "ICM_original/"
output_folder += "feature_weighting_"+feature_weighting+"/"

dataSplitter = DataSplitter_leave_k_out(dataset_class(), k_value=1, validation_set=True)
dataSplitter.load_data()

all_available_ICM_names = dataSplitter.get_loaded_ICM_names()

print("Available ICM: ", all_available_ICM_names)

ICM_name = all_available_ICM_names[0]


In [ ]:
print("Processing ICM: '{}'".format(ICM_name))

dataset_object = dataset_class(ICM_to_load_list = [ICM_name])

dataSplitter = DataSplitter_leave_k_out(dataset_object, k_value=1, validation_set=True)
dataSplitter.load_data()

ICM_object = dataSplitter.get_ICM_from_name(ICM_name)

In [ ]:
ICM_object
np.unique(ICM_object.data) # NB: 0 always included, cause ICM is sparse

In [ ]:
URM_train, URM_validation, URM_test = dataSplitter.get_holdout_split()

In [ ]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[5, 10, 15, 20, 25, 30])

In [ ]:
recommender_class = ItemKNNCBFRecommender
recommender_parameters = SearchInputRecommenderParameters(
    CONSTRUCTOR_POSITIONAL_ARGS = [ICM_object, URM_train],
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {
        'topK': 50,
        'shrink': 100, 
        'similarity': 'cosine',
        'normalize': True,
        'feature_weighting': feature_weighting,
        'ICM_bias': ICM_bias,
        #**similarity_args
    }
)

In [ ]:
start_time = time.time()

# Construct a new recommender instance
recommender_instance = recommender_class(*recommender_parameters.CONSTRUCTOR_POSITIONAL_ARGS,
                                         **recommender_parameters.CONSTRUCTOR_KEYWORD_ARGS)

recommender_instance.fit(*recommender_parameters.FIT_POSITIONAL_ARGS,
                         **recommender_parameters.FIT_KEYWORD_ARGS)
#                          **current_fit_parameters,
#                          **self.hyperparams_single_value)

train_time = time.time() - start_time
start_time = time.time()

In [ ]:
# Evaluate recommender and get results for the first cutoff
evaluation_result_dict, evaluation_result_string = evaluator_validation.evaluateRecommender(recommender_instance)
# evaluation_result_dict = evaluation_result_dict[list(evaluation_result_dict.keys())[0]]

evaluation_time = time.time() - start_time

In [ ]:
# Evaluate recommender and get results for the first cutoff
test_result_dict, test_result_string = evaluator_test.evaluateRecommender(recommender_instance)
# test_result_dict = test_result_dict[list(test_result_dict.keys())[0]]

test_time = time.time() - start_time

In [ ]:
print('Train time: ', train_time)
print()
print('Validation time: ', evaluation_time)
print('Validation result:', evaluation_result_string)
print('Test time: ', test_time)
print('Test result:', test_result_string)